In [1]:
import pandas as pd
import mysql.connector 

# Establish a connection to the MySQL database
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Haldiron123!',
    database='sakila'
)


query_may = '''
    SELECT customer_id, COUNT(*) AS rental_count
    FROM rentals_may
    GROUP BY customer_id;
'''

query_june = '''
    SELECT customer_id, COUNT(*) AS rental_count
    FROM rentals_june
    GROUP BY customer_id;
'''

df_may = pd.read_sql(query_may, con=db_connection)
df_june = pd.read_sql(query_june, con=db_connection)

db_connection.close()

print("Rentals for May:")
print(df_may)

print("\nRentals for June:")
print(df_june)


Rentals for May:
     customer_id  rental_count
0            130             4
1            459             2
2            408             6
3            333             2
4            222            10
..           ...           ...
515          191             4
516          351             2
517           10             2
518          136             2
519           61             2

[520 rows x 2 columns]

Rentals for June:
     customer_id  rental_count
0            416            10
1            516            12
2            239            10
3            285             6
4            310            12
..           ...           ...
585          412             2
586          335             2
587          226             4
588           22             2
589          126             2

[590 rows x 2 columns]


C:\Users\jseab\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\jseab\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
def compare_rentals(may_df, june_df):
    result = []
    
    for index, row in may_df.iterrows():
        customer_id = row['customer_id']
        may_rentals = row['rental_count']
        
        june_rentals = june_df[june_df['customer_id'] == customer_id]['rental_count'].values
        if len(june_rentals) == 0:
            june_rentals = 0
        else:
            june_rentals = june_rentals[0]
        
        if june_rentals > may_rentals:
            comparison = 'borrowed more'
        elif june_rentals < may_rentals:
            comparison = 'borrowed fewer'
        else:
            comparison = 'borrowed the same'
        
        result.append({
            'customer_id': customer_id,
            'comparison': comparison
        })
    
    comparison_df = pd.DataFrame(result)
    return comparison_df

comparison_result = compare_rentals(df_may, df_june)
print(comparison_result)


     customer_id         comparison
0            130      borrowed more
1            459      borrowed more
2            408  borrowed the same
3            333      borrowed more
4            222     borrowed fewer
..           ...                ...
515          191      borrowed more
516          351      borrowed more
517           10      borrowed more
518          136  borrowed the same
519           61     borrowed fewer

[520 rows x 2 columns]


In [4]:
def compare_rentals(may_df, june_df):
    
    merged_df = may_df.merge(june_df, on='customer_id', suffixes=('_may', '_june'))

    
    merged_df['comparison'] = 'borrowed the same'
    merged_df.loc[merged_df['rental_count_june'] > merged_df['rental_count_may'], 'comparison'] = 'borrowed more'
    merged_df.loc[merged_df['rental_count_june'] < merged_df['rental_count_may'], 'comparison'] = 'borrowed fewer'
    
    
    comparison_df = merged_df[['customer_id', 'comparison']]
    return comparison_df
comparison_result = compare_rentals(df_may, df_june)
print(comparison_result)


     customer_id         comparison
0            130      borrowed more
1            459      borrowed more
2            408  borrowed the same
3            333      borrowed more
4            222     borrowed fewer
..           ...                ...
507          431      borrowed more
508          191      borrowed more
509          351      borrowed more
510           10      borrowed more
511          136  borrowed the same

[512 rows x 2 columns]
